In [1]:
input_file = "gtex_rnaseq_prep_profiles.tsv"
n = 1000
pivot_url = "http://34.205.23.216:9191"

In [2]:
import livy, pprint, requests, time, json

In [3]:
host = livy.getHostUrl(pivot_url)

In [ ]:
session_url = livy.openSession(host)

.

In [ ]:
upload_file_url = session_url + "/upload-file"

In [ ]:
requests.post(upload_file_url, files={"file": open(input_file,"rb")})

In [ ]:
def code1(input_file, n):
    return '''
from pyspark import SparkContext
from pyspark import SparkFiles
from pyspark.ml.linalg import Vectors, Matrices
from pyspark.ml.stat import Correlation
import math

input_file = ''' + json.dumps(input_file) + '''
n = ''' + json.dumps(n) + '''
path = SparkFiles.get(input_file)

dataheaderindex = 2
max_rows = n

def to_float(i):
    def f(jx):
        j, x = jx
        try:
            return float(x)
        except:
            print("WARNING: non number \\"{0}\\" at ({1}, {2}) is set to 0".format(i, j, x))
            return 0
    return f
        
        
with open(path) as inp:
    headers = inp.readline().strip().split("\t")
    # ids = []
    # features = []
    ncols = len(headers) - dataheaderindex
    nrows = 0
    data = []
    for line in inp:
        if nrows >= max_rows:
            break
        row = line.strip().split('\t')
        row2 = row[dataheaderindex:]
        if len(row2) != ncols:
            print("WARNING: number of cols doesn't match at row {0} truncated with default value 0".format(nrows))
            while len(row2) < ncols:
                row2.append("0")
            row2 = row2[:ncols] 
        # id = row[:dataheaderindex]
        feature = list(map(to_float(nrows), enumerate(row2)))
        data.append(Vectors.dense(*feature))
        nrows += 1
'''

In [ ]:
def code2(input_file, n):
    return code1(input_file, n) + '''
rdd = sc.parallelize(data)

rddi = rdd.zipWithIndex()

def dot_self(v):
    return v.dot(v)
    
AA = rdd.map(dot_self).collect()

def tanimoto(veci):
    vec, i = veci
    data2 = [ABij / (AA[i] + AA[j] - ABij) for j, ABij in enumerate(map(lambda vec2 : vec.dot(vec2), data[:i]))]
    return data2
  
tanimotodf = rddi.map(tanimoto).collect()
%table tanimotodf
'''

In [ ]:
t0 = time.time()
r = livy.execStatement(host, session_url, code2(input_file, n))
t1 = time.time()
print()
print(t1 - t0)

In [ ]:
pprint.pprint(r.json()["output"]["data"]["application/vnd.livy.table.v1+json"]["data"][:10])

In [ ]:
def code3(input_file, n):
    return code1(input_file, n) + '''
data2 = list(map(lambda x : [Vectors.dense(*x)], zip(*data)))

df = spark.createDataFrame(data2, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\\n" + str(r1))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\\n" + str(r2))
'''

In [ ]:
t0 = time.time()
r = livy.execStatement(host, session_url, code3(input_file, n))
t1 = time.time()
print()
print(t1 - t0)

In [ ]:
pprint.pprint(r.json()["output"]["data"])

In [ ]:
livy.closeSession(session_url)